### Run the following cell only once after kernel started

In [1]:
import os
os.chdir('../')

### Make sure you are in "ondewo-t2s-client-python" folder

In [3]:
os.getcwd()

'/home/aryskin/ondewo/ondewo-t2s'

In [4]:
import io
import soundfile as sf
import IPython.display as ipd
import grpc
from ondewo_grpc.ondewo.t2s import text_to_speech_pb2, text_to_speech_pb2_grpc
import google.protobuf.empty_pb2 as empty_pb2
from google.protobuf.json_format import ParseDict, MessageToDict, MessageToJson

### Set up the parameters of the grpc server. The example below is for the case when server is running locally

In [5]:
MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "localhost"
GRPC_PORT: str = "50555"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"

options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]


channel = grpc.insecure_channel(CHANNEL, options=options)
stab = text_to_speech_pb2_grpc.Text2SpeechStub(channel=channel)

### List all t2s pipelines present on the server

In [6]:
pipelines = stab.ListT2sPipelines(request=empty_pb2.Empty()).pipelines

### Select pipelines for specific language language

In [7]:
def find_pipeline_for_language(pipelines, language):
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

In [8]:
english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

### Make synthesize request to the server to get audio for given text

In [37]:
request = text_to_speech_pb2.SynthesizeRequest(text='Josephus beschreibt die Bauten.Nach Josephus war die Belagerungsrampe nach ihrer Fertigstellung. mehr und mehr text. und noch ein bisschen. Und noch und noch und noch. hoch fund. 1 2 3 4 5 554 768 987 9899 76876', t2s_pipeline_id=german_pipeline.id, length_scale = 1.0, pcm=0, audio_format= 0)
response = stab.Synthesize(request=request)
print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio, )

ipd.Audio(audio[0], rate=audio[1])

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: Timer is running. Use .stop() to stop it"
	debug_error_string = "{"created":"@1612358845.669149883","description":"Error received from peer ipv4:127.0.0.1:50555","file":"src/core/lib/surface/call.cc","file_line":1061,"grpc_message":"Exception calling application: Timer is running. Use .stop() to stop it","grpc_status":2}"
>

In [38]:
request = text_to_speech_pb2.SynthesizeRequest(text='Masada ist eine archäologische Stätte in Israel. Auf einem Gipfelplateau am Rand der Judäischen Wüste', t2s_pipeline_id=english_pipeline.id, length_scale = 1.0)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 5.03328800201416 sec. Generation time is 0.2541275918483734 sec.


### Adding length scale parameter to make speech faster or slower

In [11]:
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', t2s_pipeline_id=english_pipeline.id, length_scale=0.5)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 0.7778685092926025 sec. Generation time is 0.11872763186693192 sec.


### Get pipeline you want to update

In [35]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)
pipeline_config = stab.GetT2sPipeline(request=request)

### Change parameter in the pipeline config. For example default length scale

In [36]:
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 2

### Update pipeline

In [37]:
stab.UpdateT2sPipeline(request=pipeline_config)

### See if generated audio change according to updated config

In [38]:
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', t2s_pipeline_id=english_pipeline.id)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 2.9605441093444824 sec. Generation time is 0.17088665068149567 sec.


### Change parameter back to previous (length_scale = 1.0)

In [39]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)
pipeline_config = stab.GetT2sPipeline(request=request)
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 1.0
stab.UpdateT2sPipeline(request=pipeline_config)